In [9]:
import numpy as np
import pandas as pd
import copy
from stA_star import *
from map import *



In [ ]:
class GeneticAlgorithm:
    def __init__(self, population_size, mutation_rate, crossover_rate, sku_info):
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.sku_info = sku_info
        self.population = self.initialize_population()

    def initialize_population(self):
        # 初始化种群
        population = []
        for _ in range(self.population_size):
            chromosome = self.generate_chromosome_with_check(self.sku_info)
            population.append(chromosome)
        return population

    def generate_chromosome_with_check(self,sku_info):
        # 生成随机基因
        global demand
        tmp = copy.deepcopy(demand)
        chromosome = []
        for i in range(len(sku_info)):
            goal_idx = np.where(demand[:,sku_info[i][0]]>0)[0]
            if goal_idx.shape[0] > 0:
                gene = int(np.random.choice(goal_idx))
                chromosome.append(gene)
                tmp[int(gene)][sku_info[i][0]] -= 1
            else:
                gene = -1
                chromosome.append(gene)
        return chromosome
        
    def evaluate_fitness(self, chromosome):
       # 评价适应度
        entrances = {0:(0,0),1:(0,3),2:(0,6),3:(9,1),4:(9,4),5:(9,7)}
        goal = {0:(3,2),1:(3,5),2:(6,2),3:(6,5)}
        start =[]
        end = []
        for i in range(len(self.sku_info)):
            start.append(entrances[self.sku_info[i][2]] + (self.sku_info[i][1],))
        for i in range(len(chromosome)):
            if chromosome[i] != -1:
                end.append(goal[chromosome[i]])
            else:
                end.append((-1,-1,-1))
        ST_Table = np.expand_dims(raw_map.map, axis=0)
        fitness = 0
        for i in range(len(start)):
            road, work_time=StAstar(raw_map.map,start[i],end[i],ST_Table)
            #print(f'商品{j}最短路径：',road,'用时：',work_time+1)
            ST_Table = update_StTable(raw_map.map,ST_Table,road[:-1])
            fitness += (work_time+1)
        return -fitness

    def roulette_wheel_selection(self):
        # 轮盘赌选择父代
        fitness_values = [self.evaluate_fitness(individual) for individual in self.population]

        # 将适应度转换为正数（假设更接近0的适应度更好）
        min_fitness = min(fitness_values)
        adjusted_fitness = [fitness + abs(min_fitness) + 1e-4 for fitness in fitness_values]  # 转换为正数，越接近0的适应度变得越大
        # 计算调整后的总适应度
        total_fitness = sum(adjusted_fitness)

        # 计算每个个体的选择概率
        selection_probabilities = [fitness / total_fitness for fitness in adjusted_fitness]
        
        # 选择父代
        population_indices = list(range(len(self.population)))
        selected_indices = random.choices(population_indices, weights=selection_probabilities, k=2)
        parents = [self.population[index] for index in selected_indices]

        return parents

    def crossover(self, parent1, parent2):
        # 单点交叉
        global demand
        tmp = copy.deepcopy(demand)
        check = np.zeros(demand.shape)
        crossover_point = random.randint(0, len(parent1) - 1)
        offspring1 = parent1[:crossover_point] + parent2[crossover_point:]
        offspring2 = parent2[:crossover_point] + parent1[crossover_point:]
        # 检查是否满足需求
        for i in range(len(offspring1)):
            if offspring1[i] != -1:
                check[int(offspring1[i])][sku_info[i][0]] += 1
        wrong_demand = demand - check
        for i in range(max(self.sku_info[:,0])):
            na_demand_idx = np.where(wrong_demand[:,i] < 0)[0]
            po_demand_idx = np.where(wrong_demand[:,i] > 0)[0]
            sku_idx = np.where(self.sku_info[:,0] == i)[0]
            while len(na_demand_idx) > 0:
                for j in sku_idx:
                    if offspring1[j] in na_demand_idx:
                        fix_pos = np.random.choice(po_demand_idx)
                        wrong_demand[offspring1[j]][i] += 1
                        wrong_demand[fix_pos][i] -= 1
                        offspring1[j] = fix_pos
                        na_demand_idx = np.where(wrong_demand[:,i] < 0)[0]
                        po_demand_idx = np.where(wrong_demand[:,i] > 0)[0]
                        break
        check = np.zeros(demand.shape)
        for i in range(len(offspring2)):
            if offspring2[i] != -1:
                check[int(offspring2[i])][sku_info[i][0]] += 1
        wrong_demand = demand - check
        for i in range(max(self.sku_info[:,0])):
            na_demand_idx = np.where(wrong_demand[:,i] < 0)[0]
            po_demand_idx = np.where(wrong_demand[:,i] > 0)[0]
            sku_idx = np.where(self.sku_info[:,0] == i)[0]
            while len(na_demand_idx) > 0:
                for j in sku_idx:
                    if offspring2[j] in na_demand_idx:
                        fix_pos = np.random.choice(po_demand_idx)
                        wrong_demand[offspring2[j]][i] += 1
                        wrong_demand[fix_pos][i] -= 1
                        offspring2[j] = fix_pos
                        na_demand_idx = np.where(wrong_demand[:,i] < 0)[0]
                        po_demand_idx = np.where(wrong_demand[:,i] > 0)[0]
                        break
        return offspring1, offspring2
    def mutate(self, chromosome):
        # Mutate a chromosome by randomly changing its genes
        for i in range(len(chromosome)):
            if random.random() < self.mutation_rate:
                chromosome[i] = random.randint(0, len(chromosome) - 1)
        return chromosome
    
    def select_best_individuals(self, offspring):
        # Select the best individuals for the next generation
        fitness = []
        for i in range(len(offspring)):
            fitness.append(self.evaluate_fitness(offspring[i]))
        fitness = np.array(fitness)
        idx = np.argsort(fitness)
        return [offspring[i] for i in idx[:self.population_size]]
    

    def run(self, generations):
    # 运行遗传算法指定的代数
        best_fitness = -100000
        for gen in range(generations):
            # 初始化下一代的后代列表
            next_generation = []

            # 继续选择和交叉，直到达到种群大小
            while len(next_generation) < self.population_size:
                # 从当前种群中选择父代
                parents = self.roulette_wheel_selection()
                # 对每对父代进行交叉
                for i in range(0, len(parents), 2):
                    parent1 = parents[i]
                    parent2 = parents[min(i + 1, len(parents) - 1)]  # 确保不越界
                    if random.random() < self.crossover_rate:
                        offspring1, offspring2 = self.crossover(parent1, parent2)
                        next_generation.extend([offspring1, offspring2])
                    else:
                        next_generation.extend([parent1, parent2])

                    # 如果已经生成足够数量的后代，就跳出循环
                    if len(next_generation) >= self.population_size:
                        break

            '''# 变异
            for i in range(len(next_generation)):
                next_generation[i] = self.mutate(next_generation[i])'''

            # 评估后代的适应度，并选择下一代
            self.population = self.select_best_individuals(next_generation)

            # 输出当前的最优解
            '''best_chromosome = self.population[0]
            best_fitness = self.evaluate_fitness(best_chromosome)
            print(f'第{gen+1}代的最优解：', best_chromosome, '适应度：', -best_fitness)'''
            if self.evaluate_fitness(self.population[0]) > best_fitness:
                best_chromosome = self.population[0]
                best_fitness = self.evaluate_fitness(best_chromosome)
            print(f'第{gen+1}代的最优解：', best_chromosome, '适应度：', -best_fitness)

        return best_chromosome, best_fitness

# 测试迭代
if __name__ == '__main__':
    file_path = "测试小算例.xlsx"
    sku_data = load_data(file_path,sheet_name="sku_info")
    sku_info = np.array(sku_data.iloc[:,1:])
    demand = np.array(load_data(file_path,sheet_name="demand"))
    raw_map = Map()  
    population_size = 50
    mutation_rate = 0.1
    crossover_rate = 0.65
    generations = 60
    genetic_algorithm = GeneticAlgorithm(population_size, mutation_rate, crossover_rate, sku_info)
    best_chromosome, best_fitness = genetic_algorithm.run(generations)
    

In [3]:
def load_data(file_path,sheet_name):
    data =pd.read_excel(file_path,sheet_name=sheet_name,header=None)
    return data[1:]

def load_map(map_path):
    data =pd.read_excel(map_path,header=None)
    map = np.array(data)
    return map

file_path = "CAsimulator\instance\s_1.xlsx"
map_path = "map.xlsx"
sku_data = load_data(file_path,sheet_name="sku_info")
sku_info = np.array(sku_data.iloc[:,1:])
demand = np.array(load_data(file_path,sheet_name="demand"))
map = load_map(map_path)

In [4]:
map

array([[2, 3, 5, 2, 3, 5, 2, 3],
       [0, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 1, 0, 0],
       [1, 1, 4, 1, 1, 4, 1, 1],
       [0, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 1, 0, 0],
       [1, 1, 4, 1, 1, 4, 1, 1],
       [0, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 1, 0, 0],
       [3, 2, 5, 3, 2, 5, 3, 2]], dtype=int64)

In [5]:
sku_data

,0,1,2,3
1,黄瓜 1kg±0.1kg,3,0,3
2,南丰蜜桔 500g ±50g,4,0,3
3,士力架花生 51g/条,2,0,2
4,黄瓜 1kg±0.1kg,3,0,3
5,豆王 300g±30g,1,0,1
6,士力架花生 51g/条,2,0,1
7,豆王 300g±30g,1,1,0
8,南丰蜜桔 500g ±50g,4,1,5
9,南丰蜜桔 500g ±50g,4,1,3
10,秀珍菇 300g±30g/份,0,2,1


In [6]:
sku_info #sku的种类， 到达时间， 入口

array([[3, 0, 3],
       [4, 0, 3],
       [2, 0, 2],
       [3, 0, 3],
       [1, 0, 1],
       [2, 0, 1],
       [1, 1, 0],
       [4, 1, 5],
       [4, 1, 3],
       [0, 2, 1],
       [0, 2, 0],
       [4, 3, 5],
       [4, 3, 5],
       [4, 3, 0],
       [4, 3, 3],
       [4, 3, 4],
       [2, 3, 1],
       [3, 4, 2],
       [3, 4, 3],
       [0, 5, 2],
       [2, 6, 2],
       [3, 7, 5],
       [4, 7, 5],
       [4, 7, 5],
       [3, 7, 2],
       [2, 8, 3],
       [3, 9, 4],
       [4, 10, 3],
       [0, 10, 1]], dtype=object)

In [69]:
demand #投递口，sku，需求量

array([[13, 12, 14, 15, 16],
       [18, 18, 14, 13, 11],
       [17, 16, 19, 12, 10],
       [15, 12, 17, 11, 11]], dtype=object)

In [70]:
map

array([[2, 3, 5, 2, 3, 5, 2, 3],
       [0, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 1, 0, 0],
       [1, 1, 4, 1, 1, 4, 1, 1],
       [0, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 1, 0, 0],
       [1, 1, 4, 1, 1, 4, 1, 1],
       [0, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 1, 0, 0],
       [3, 2, 5, 3, 2, 5, 3, 2]], dtype=int64)

In [81]:
entrances = [i for i in zip(np.where(map==2)[0],np.where(map==2)[1])]
deliverPos = [i for i in zip(np.where(map==4)[0],np.where(map==4)[1])]
entrances_dic = {i:entrances[i] for i in range(len(entrances))}
deliverPos_dic = {i:deliverPos[i] for i in range(len(deliverPos))}
ST_Table = np.expand_dims(map, axis=0)

# 贪婪搜索，根据sku的到达入口，找到有需求的最近的投递口
def greedy_search(sku_info, demand, map):
    solution = []
    for sku in range(len(sku_info)):
        sku_pos = entrances_dic[sku_info[sku][2]]
        sku_type = sku_info[sku][0]
        sku_demand = demand[:,sku_type]
        # 返回有需求的投递口
        sku_demand_idx = np.where(sku_demand>0)[0]
        # 计算距离
        distance2deliverPos = {}
        for i in sku_demand_idx:
            distance2deliverPos[i] = abs(sku_pos[0]-deliverPos_dic[i][0])+abs(sku_pos[1]-deliverPos_dic[i][1])
        # 返回最近的投递口
        min_distance_pos = min(distance2deliverPos, key=distance2deliverPos.get)
        # 更新需求
        demand[min_distance_pos][sku_type] -= 1
        solution.append(min_distance_pos)
    return solution, demand

In [77]:
solution, demand_new = greedy_search(sku_info, copy.deepcopy(demand), map)

In [78]:
demand

array([[13, 12, 14, 15, 16],
       [18, 18, 14, 13, 11],
       [17, 16, 19, 12, 10],
       [15, 12, 17, 11, 11]], dtype=object)

In [79]:
demand_new

array([[10, 10, 12, 15, 15],
       [17, 18, 12, 11, 11],
       [17, 16, 18, 9, 6],
       [15, 12, 17, 9, 5]], dtype=object)

In [80]:
print(solution)

[2, 3, 3, 0, 2, 3, 3, 0, 2, 3, 0, 1, 3, 2, 1, 0, 3, 1, 2, 3, 2, 1, 2, 0, 0, 1, 0, 0, 2]


In [83]:
def calculate_time(solution):
    global ST_Table
    start =[]
    end = []
    for i in range(len(sku_info)):
        start.append(entrances_dic[sku_info[i][2]] + (sku_info[i][1],))
    for i in range(len(solution)):
        end.append(deliverPos_dic[solution[i]])
    time = 0
    for i in range(len(start)):
        road, work_time=StAstar(map,start[i],end[i],ST_Table)
        print(f'商品{i}最短路径：',road,'用时：',work_time+1)
        ST_Table = update_StTable(map,ST_Table,road[:-1])
        time += (work_time+1)
    return time

In [84]:
calculate_time(solution)

商品0最短路径： [(9, 1, 2), (8, 1, 3), (7, 1, 4), (6, 1, 5), (6, 2)] 用时： 4
商品1最短路径： [(9, 4, 3), (8, 4, 4), (7, 4, 5), (6, 4, 6), (6, 5)] 用时： 4
商品2最短路径： [(9, 7, 6), (8, 7, 7), (7, 7, 8), (6, 7, 9), (5, 7, 10), (5, 7, 11), (5, 6, 12), (5, 5, 13), (6, 5)] 用时： 8
商品3最短路径： [(0, 0, 6), (1, 0, 7), (2, 0, 8), (3, 0, 9), (4, 0, 10), (4, 0, 11), (4, 1, 12), (4, 2, 13), (3, 2)] 用时： 8
商品4最短路径： [(9, 1, 11), (8, 1, 12), (7, 1, 13), (6, 1, 14), (6, 2)] 用时： 4
商品5最短路径： [(9, 7, 12), (8, 7, 13), (7, 7, 14), (6, 7, 15), (5, 7, 16), (5, 7, 17), (5, 6, 18), (5, 5, 19), (6, 5)] 用时： 8
商品6最短路径： [(9, 7, 14), (8, 7, 15), (7, 7, 16), (6, 7, 17), (5, 7, 18), (5, 7, 19), (5, 6, 20), (5, 5, 21), (6, 5)] 用时： 8
商品7最短路径： [(0, 0, 20), (1, 0, 21), (2, 0, 22), (3, 0, 23), (4, 0, 24), (4, 0, 25), (4, 1, 26), (4, 2, 27), (3, 2)] 用时： 8
商品8最短路径： [(9, 1, 21), (8, 1, 22), (7, 1, 23), (6, 1, 24), (6, 2)] 用时： 4
商品9最短路径： [(9, 7, 30), (8, 7, 31), (7, 7, 32), (6, 7, 33), (5, 7, 34), (5, 7, 35), (5, 6, 36), (5, 5, 37), (6, 5)] 用时： 8
商品10最短路径

152

In [86]:
ST_Table.shape

(64, 10, 8)

In [87]:
sku_info

array([[3, 2, 3],
       [3, 3, 4],
       [4, 6, 5],
       [1, 6, 0],
       [4, 11, 3],
       [4, 12, 5],
       [3, 14, 5],
       [4, 20, 0],
       [4, 21, 3],
       [4, 30, 5],
       [0, 33, 1],
       [2, 34, 2],
       [4, 38, 4],
       [4, 38, 3],
       [2, 39, 2],
       [2, 39, 1],
       [4, 39, 5],
       [0, 40, 2],
       [4, 43, 3],
       [4, 45, 5],
       [2, 47, 3],
       [3, 47, 2],
       [3, 51, 3],
       [1, 52, 1],
       [0, 53, 1],
       [3, 54, 2],
       [0, 56, 0],
       [2, 57, 1],
       [3, 59, 3]], dtype=object)

In [7]:
sku_info[:,1]

array([0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 5, 6, 7,
       7, 7, 7, 8, 9, 10, 10], dtype=object)

In [30]:
import random

def generate_random_numbers(target_sum, num_cells):
    while True:
        numbers = [random.randint(0, target_sum) for _ in range(num_cells - 1)]
        numbers.append(target_sum - sum(numbers))

        if all(0 <= n <= target_sum for n in numbers):
            return numbers

# 生成总和为6的4个随机正整数或0
random_numbers = generate_random_numbers(19, 4)
random_numbers


[9, 1, 4, 5]

In [15]:
def evaluate_fitness(chromosome):
       # 评价适应度
        entrances = {0:(0,0),1:(0,3),2:(0,6),3:(9,1),4:(9,4),5:(9,7)}
        goal = {0:(3,2),1:(3,5),2:(6,2),3:(6,5)}
        start =[]
        end = []
        for i in range(len(sku_info)):
            start.append(entrances[sku_info[i][2]] + (sku_info[i][1],))
        for i in range(len(chromosome)):
            if chromosome[i] != -1:
                end.append(goal[chromosome[i]])
            else:
                end.append((-1,-1,-1))
        ST_Table = np.expand_dims(raw_map.map, axis=0)
        fitness = 0
        for i in range(len(start)):
            road, work_time=StAstar(raw_map.map,start[i],end[i],ST_Table)
            print(f'商品{i}最短路径：',road,'用时：',work_time+1)
            ST_Table = update_StTable(raw_map.map,ST_Table,road[:-1])
            fitness += (work_time+1)
        return fitness

def load_data(file_path,sheet_name):
    data =pd.read_excel(file_path,sheet_name=sheet_name,header=None)
    return data[1:]

def load_map(map_path):
    data =pd.read_excel(map_path,header=None)
    map = np.array(data)
    return map

In [16]:
file_path = "CAsimulator\instance\s_1.xlsx"
map_path = "map.xlsx"
sku_data = load_data(file_path,sheet_name="sku_info")
sku_info = np.array(sku_data.iloc[:,1:])
demand = np.array(load_data(file_path,sheet_name="demand"))
raw_map = Map()

In [18]:
evaluate_fitness([3, 3, 3, 0, 2, 1, 1, 2, 0, 2, 2, 2, 2, 1, 2])

商品0最短路径： [(9, 1, 0), (8, 1, 1), (7, 1, 2), (7, 1, 3), (7, 2, 4), (7, 3, 5), (7, 4, 6), (7, 5, 7), (6, 5)] 用时： 8
商品1最短路径： [(0, 6, 0), (1, 6, 1), (2, 6, 2), (3, 6, 3), (4, 6, 4), (5, 6, 5), (6, 6, 6), (6, 5)] 用时： 7
商品2最短路径： [(0, 3, 0), (1, 3, 1), (2, 3, 2), (3, 3, 3), (4, 3, 4), (5, 3, 5), (6, 3, 6), (7, 3, 7), (7, 3, 8), (7, 4, 9), (7, 5, 10), (6, 5)] 用时： 11
商品3最短路径： [(0, 3, 1), (1, 3, 2), (2, 3, 3), (3, 3, 4), (3, 2)] 用时： 4
商品4最短路径： [(9, 1, 2), (8, 1, 3), (7, 1, 4), (6, 1, 5), (6, 2)] 用时： 4
商品5最短路径： [(0, 0, 3), (1, 0, 4), (2, 0, 5), (3, 0, 6), (4, 0, 7), (4, 0, 8), (4, 1, 9), (4, 2, 10), (4, 3, 11), (4, 4, 12), (4, 5, 13), (3, 5)] 用时： 11
商品6最短路径： [(9, 7, 3), (8, 7, 4), (7, 7, 5), (6, 7, 6), (5, 7, 7), (4, 7, 8), (3, 7, 9), (2, 7, 10), (2, 7, 11), (2, 6, 12), (2, 5, 13), (3, 5)] 用时： 11
商品7最短路径： [(9, 1, 4), (8, 1, 5), (7, 1, 6), (6, 1, 7), (6, 2)] 用时： 4
商品8最短路径： [(0, 3, 5), (1, 3, 6), (2, 3, 7), (3, 3, 8), (3, 2)] 用时： 4
商品9最短路径： [(0, 0, 5), (1, 0, 6), (2, 0, 7), (3, 0, 8), (4, 0, 9), (5,

115

In [19]:
evaluate_fitness([2, 3, 1, 0, 2, 3, 1, 2, 0, 2, 3, 1, 2, 2, 2])

商品0最短路径： [(9, 1, 0), (8, 1, 1), (7, 1, 2), (6, 1, 3), (6, 2)] 用时： 4
商品1最短路径： [(0, 6, 0), (1, 6, 1), (2, 6, 2), (3, 6, 3), (4, 6, 4), (5, 6, 5), (6, 6, 6), (6, 5)] 用时： 7
商品2最短路径： [(0, 3, 0), (1, 3, 1), (2, 3, 2), (3, 3, 3), (4, 3, 4), (4, 3, 5), (4, 4, 6), (4, 5, 7), (3, 5)] 用时： 8
商品3最短路径： [(0, 3, 1), (1, 3, 2), (2, 3, 3), (3, 3, 4), (3, 2)] 用时： 4
商品4最短路径： [(9, 1, 2), (8, 1, 3), (7, 1, 4), (6, 1, 5), (6, 2)] 用时： 4
商品5最短路径： [(0, 0, 3), (1, 0, 4), (2, 0, 5), (3, 0, 6), (4, 0, 7), (5, 0, 8), (6, 0, 9), (7, 0, 10), (7, 0, 11), (7, 1, 12), (7, 2, 13), (7, 3, 14), (7, 4, 15), (7, 5, 16), (6, 5)] 用时： 14
商品6最短路径： [(9, 7, 3), (8, 7, 4), (7, 7, 5), (6, 7, 6), (5, 7, 7), (4, 7, 8), (3, 7, 9), (2, 7, 10), (2, 7, 11), (2, 6, 12), (2, 5, 13), (3, 5)] 用时： 11
商品7最短路径： [(9, 1, 4), (8, 1, 5), (7, 1, 6), (6, 1, 7), (6, 2)] 用时： 4
商品8最短路径： [(0, 3, 5), (1, 3, 6), (2, 3, 7), (3, 3, 8), (3, 2)] 用时： 4
商品9最短路径： [(0, 0, 5), (1, 0, 6), (2, 0, 7), (3, 0, 8), (4, 0, 9), (5, 0, 10), (6, 0, 11), (7, 0, 12), (7, 0, 13)

118

In [41]:
A = [3.7608066779892017e-08, 0.0026326022826592206, 0.010530296306436544, 0.011658538309833304, 0.013915022316626826, 0.015419344987822508, 0.015795425655621427, 0.016547586991219266, 0.016547586991219266, 0.01729974832681711, 0.01729974832681711, 0.01767582899461603, 0.01842799033021387, 0.01880407099801279, 0.01880407099801279, 0.01955623233361063, 0.01993231300140955, 0.020308393669208467, 0.02106055500480631, 0.02294095834380091, 0.02444528101499659, 0.025197442350594432, 0.02820608769298579, 0.028582168360784713, 0.028958249028583634, 0.029710410364181473, 0.029710410364181473, 0.03234297503877391, 0.033847297709969595, 0.03459945904556744, 0.03535162038116527, 0.03535162038116527, 0.03647986238456204, 0.03723202372015987, 0.03760810438795879, 0.03911242705915448, 0.039864588394752314, 0.04174499173374692, 0.04174499173374692, 0.044753637076138275]
sum(A)

0.9999999999999998

In [38]:
import random
random.random()

0.4132460688505111

In [45]:
random.randint(0,5)

3

In [61]:
import numpy as np
a = np.zeros((5,3))
np.where(a[:,0]>0)[0].shape[0]

0